# ADA 2018 - Homework 3



## Undestanding the StackOverflow community


Deadline: Nov 7th 2018, 23:59:59

Submission link: Check channel homework-3-public

StackOverflow is the most popular programming-related Q&A website. It serves as a platform for users to ask and answer questions and to vote questions and answers up or down. Users of StackOverflow can earn reputation points and "badges"; for example, a person is awarded 10 reputation points for receiving an "up" vote on an answer given to a question, and 5 points for the "up" vote on a question asked. Also, users receive badges for their valued contributions, which represents a kind of gamification of the traditional Q&A site. 

[Learn more about StackOverflow on Wikipedia](https://en.wikipedia.org/wiki/Stack_Overflow)

----

Dataset link:

https://drive.google.com/open?id=1POlGjqzw9v_pZ_bUnXGihOgk45kbvNjB

http://iccluster053.iccluster.epfl.ch/Posts.json.zip (mirror 1)

https://iloveadatas.com/datasets/Posts.json.zip (mirror 2)

Dataset description:

* **Id**: Id of the post
* **CreationDate**: Creation date of the post (String format)
* **PostTypeId**: Type of post (Question = 1, Answer = 2)
* **ParentId**: The id of the question. Only present if PostTypeId = 2
* **Score**: Points assigned by the users
* **Tags**: Tags of the question. Only present if PostTypeId = 1
* **Title**: Only present if PostTypeId = 1
* **ViewCount**: Only present if PostTypeId = 1

The dataset format is JSON. Here are examples of a question and an answer:

Question:
```json
{
    "Id": 10130734,
    "CreationDate": "2012-04-12T19:51:25.793+02:00",
    "PostTypeId": 1,
    "Score": 4,
    "Tags": "<python><pandas>",
    "Title": "Best way to insert a new value",
    "ViewCount": 3803
}
```

Answer:
```json
{  
   "CreationDate":"2010-10-26T03:19:05.063+02:00",
   "Id":4020440,
   "ParentId":4020214,
   "PostTypeId":2,
   "Score":1
}
```

----
Useful resources:

**Spark SQL, DataFrames and Datasets Guide**

https://spark.apache.org/docs/latest/sql-programming-guide.html

**Database schema documentation for the public data dump**

https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

----

**Note:** Use Spark where possible. Some computations can take more than 10 minutes on a common notebook. Consider to save partial results on disk.

In [35]:
import re

import os
import shutil

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import unix_timestamp, udf, to_date, to_timestamp
from pyspark.sql.types import *
from datetime import datetime

EVENTS_SCHEMA = StructType([
    StructField("GLOBALEVENTID", LongType(), True),
    StructField("Day_DATE", StringType(), True),
    StructField("MonthYear_Date", StringType(), True),
    StructField("Year_Date", StringType(), True),
    StructField("FractionDate", FloatType(), True),
    StructField("Actor1Code", StringType(), True),
    StructField("Actor1Name", StringType(), True),
    StructField("Actor1CountryCode", StringType(), True),
    StructField("Actor1KnownGroupCode", StringType(), True),
    StructField("Actor1EthnicCode", StringType(), True),
    StructField("Actor1Religion1Code", StringType(), True),
    StructField("Actor1Religion2Code", StringType(), True),
    StructField("Actor1Type1Code", StringType(), True),
    StructField("Actor1Type2Code", StringType(), True),
    StructField("Actor1Type3Code", StringType(), True),
    StructField("Actor2Code", StringType(), True),
    StructField("Actor2Name", StringType(), True),
    StructField("Actor2CountryCode", StringType(), True),
    StructField("Actor2KnownGroupCode", StringType(), True),
    StructField("Actor2EthnicCode", StringType(), True),
    StructField("Actor2Religion1Code", StringType(), True),
    StructField("Actor2Religion2Code", StringType(), True),
    StructField("Actor2Type1Code", StringType(), True),
    StructField("Actor2Type2Code", StringType(), True),
    StructField("Actor2Type3Code", StringType(), True),
    StructField("IsRootEvent", LongType(), True),
    StructField("EventCode", StringType(), True),
    StructField("EventBaseCode", StringType(), True),
    StructField("EventRootCode", StringType(), True),
    StructField("QuadClass", LongType(), True),
    StructField("GoldsteinScale", FloatType(), True),
    StructField("NumMentions", LongType(), True),
    StructField("NumSources", LongType(), True),
    StructField("NumArticles", LongType(), True),
    StructField("AvgTone", FloatType(), True),
    StructField("Actor1Geo_Type", LongType(), True),
    StructField("Actor1Geo_FullName", StringType(), True),
    StructField("Actor1Geo_CountryCode", StringType(), True),
    StructField("Actor1Geo_ADM1Code", StringType(), True),
    StructField("Actor1Geo_ADM2Code", StringType(), True),
    StructField("Actor1Geo_Lat", FloatType(), True),
    StructField("Actor1Geo_Long", FloatType(), True),
    StructField("Actor1Geo_FeatureID", StringType(), True),
    StructField("Actor2Geo_Type", LongType(), True),
    StructField("Actor2Geo_FullName", StringType(), True),
    StructField("Actor2Geo_CountryCode", StringType(), True),
    StructField("Actor2Geo_ADM1Code", StringType(), True),
    StructField("Actor2Geo_ADM2Code", StringType(), True),
    StructField("Actor2Geo_Lat", FloatType(), True),
    StructField("Actor2Geo_Long", FloatType(), True),
    StructField("Actor2Geo_FeatureID", StringType(), True),
    StructField("ActionGeo_Type", LongType(), True),
    StructField("ActionGeo_FullName", StringType(), True),
    StructField("ActionGeo_CountryCode", StringType(), True),
    StructField("ActionGeo_ADM1Code", StringType(), True),
    StructField("ActionGeo_ADM2Code", StringType(), True),
    StructField("ActionGeo_Lat", FloatType(), True),
    StructField("ActionGeo_Long", FloatType(), True),
    StructField("ActionGeo_FeatureID", StringType(), True),
    StructField("DATEADDED", LongType(), True),
    StructField("SOURCEURL", StringType(), True)
])


MENTIONS_SCHEMA = StructType([
    StructField("GLOBALEVENTID", LongType(), True),
    StructField("EventTimeDate", LongType(), True),
    StructField("MentionTimeDate", LongType(), True),
    StructField("MentionType", LongType(), True),
    StructField("MentionSourceName", StringType(), True),
    StructField("MentionIdentifier", StringType(), True),
    StructField("SentenceID", LongType(), True),
    StructField("Actor1CharOffset", LongType(), True),
    StructField("Actor2CharOffset", LongType(), True),
    StructField("ActionCharOffset", LongType(), True),
    StructField("InRawText", LongType(), True),
    StructField("Confidence", LongType(), True),
    StructField("MentionDocLen", LongType(), True),
    StructField("MentionDocTone", FloatType(), True),
    StructField("MentionDocTranslationInfo", StringType(), True),
    StructField("Extras", StringType(), True)
])

OUT_DIR = 'output'
DATA_DIR = '.'


def save(df):
    df.write.mode('overwrite').csv('df_save')


spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext

events_raw = spark.read.csv(os.path.join(DATA_DIR, "*.export.CSV"), sep="\t", schema=EVENTS_SCHEMA)
mentions_raw = spark.read.csv(os.path.join(DATA_DIR, "*.mentions.CSV"), sep="\t", schema=MENTIONS_SCHEMA)


events_raw.show(5)
mentions_raw.show(5)

+-------------+--------+--------------+---------+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+----------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+
|GLOBALEVENTI

In [36]:
events = events_raw.select('GLOBALEVENTID',
                           to_date(events_raw.Day_DATE.cast('String'), 'yyyyMMdd').alias('Day_DATE'),
                           'MonthYear_Date',
                           'Year_Date',
                           'FractionDate',
                           'Actor1CountryCode',
                           'Actor2CountryCode',
                           'QuadClass',
                           'GoldsteinScale',
                           'AvgTone',
                           'Actor1Geo_CountryCode',
                           'Actor2Geo_CountryCode')


mentions = mentions_raw.select('GLOBALEVENTID',
                               to_timestamp(mentions_raw.EventTimeDate.cast('String'), 'yyyyMMddHHmmss').alias(
                                   'EventTimeDate'), 
                               to_timestamp(mentions_raw.MentionTimeDate.cast('String'), 'yyyyMMddHHmmss').alias(
                                   'MentionTimeDate'),
                               'MentionType',
                               'Confidence')

events.registerTempTable('events')
mentions.registerTempTable('mentions')

events.show(5)
mentions.show(5)

+-------------+----------+--------------+---------+------------+-----------------+-----------------+---------+--------------+----------+---------------------+---------------------+
|GLOBALEVENTID|  Day_DATE|MonthYear_Date|Year_Date|FractionDate|Actor1CountryCode|Actor2CountryCode|QuadClass|GoldsteinScale|   AvgTone|Actor1Geo_CountryCode|Actor2Geo_CountryCode|
+-------------+----------+--------------+---------+------------+-----------------+-----------------+---------+--------------+----------+---------------------+---------------------+
|    410479387|2014-02-19|        201402|     2014|   2014.1342|             null|              AUS|        1|           2.8| 2.1943574|                 null|                   AS|
|    410479388|2014-02-19|        201402|     2014|   2014.1342|             null|             null|        1|           1.9|-2.5711663|                 null|                   US|
|    410479389|2014-02-19|        201402|     2014|   2014.1342|             null|             